<a href="https://colab.research.google.com/github/rdsza/CUDAProg/blob/LinAlg/Cuda_Cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%shell
nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


# Starting simple

We'll start with a simple C++ program that adds the elements of two arrays with a million elements each

In [4]:
%%writefile add.cpp

#include <iostream>
#include <math.h>

// function to add the elements of two arrays
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20; // 1M elements

  float *x = new float[N];
  float *y = new float[N];

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the CPU
  add(N, x, y);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  delete [] x;
  delete [] y;

  return 0;
}

Writing add.cpp


In [5]:
%%shell
g++ add.cpp -o add

In [6]:
%%shell
./add

Max error: 0


As expected, it prints that there was no error in the summation and then exits. Now

```cpp
// CUDA Kernel function to add the elements of two arrays on the GPU
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}
```

These `__global__` functions are known as *kernels*, and code tat runs on the GPU is often called *device code*, while the code that runs on the CPU is *host code*.

# Memory Allocation in CUDA

To compute on the GPU, i need to allocate memory accessible by the GPU. [Unified Memory](https://developer.nvidia.com/blog/unified-memory-in-cuda-6/) in CUDA makes this easy by providing a single memory space accessible by all GPUs and CPUs in your system. To allocate data in unified memory, call `cudaMallocManaged()`, which returns a pointer that you can access from the host (CPU) code or device (GPU) code. To free the data, just pass the pointer to `cudaFree()`.

I just need to replace the calls to `new` in the code above with calls to `cudaMallocManaged()`, and replace calls to `delete []` with calls to `cudaFree`.

```cpp
// Allocate Unified Memory -- accessible from CPU or GPU
float *x, *y;
cudaMallocManaged(&x, N*sizeof(float));
cudaMallocManaged(&y, N*sizeof(float));

...

//Free memory
cudaFree(x);
cudaFree(y);
```

Finally, I need to *launch* the`add()` kernel, which invokes it on the GPU. CUDA kernel launches are specific using the triple angle bracket syntax `<<< >>>`. I just have to add it to the call to `add` before the parameter list.

```cpp
add<<<1, 1>>>(N, x, y);
```

Easy! I'll get int the details of what goes inside the angle brackets soon; for now all you need to know is that this line launches one GPU thread to run `add()`.
Just one more thing: I need the CPU to wait until the kernel is done before it accesses the results (because CUDA kernel launches don't block the calling CPU thread). To do this I just call `cudaDeviceSynchronize()` before doing the final error checking on the CPU.
Here's the complete code:

In [20]:
%%writefile add.cu

#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20
 ;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);

  return 0;
}

Writing add.cu


In [22]:
%%shell

nvcc add.cu -o add_cuda
./add_cuda

Max error: 0


This is only a first step, because as written, this kernel is only correct for a single thread, since every thread that runs it will perform the add on the whole array. Moreover, there is a race condition since multiple parallel threads would both read and write the same locations.

# Profile it!

I think the simplest way to find out how long the kernels takes to run is to run it with `nvprof`, the command line GPU profiler that comes with the CUDA Toolkit. Just type `nvprof ./add_cuda` on the command line:

In [23]:
%%shell

nvprof ./add_cuda

==1778== NVPROF is profiling process 1778, command: ./add_cuda
Max error: 0
==1778== Profiling application: ./add_cuda
==1778== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  110.48ms         1  110.48ms  110.48ms  110.48ms  add(int, float*, float*)
      API calls:   64.41%  201.75ms         2  100.87ms  41.776us  201.71ms  cudaMallocManaged
                   35.28%  110.50ms         1  110.50ms  110.50ms  110.50ms  cudaDeviceSynchronize
                    0.17%  524.49us         2  262.24us  256.42us  268.06us  cudaFree
                    0.09%  267.94us         1  267.94us  267.94us  267.94us  cudaLaunchKernel
                    0.05%  165.42us       114  1.4510us     135ns  60.072us  cuDeviceGetAttribute
                    0.00%  11.239us         1  11.239us  11.239us  11.239us  cuDeviceGetName
                    0.00%  5.0040us         1  5.0040us  5.0040us  5.0040us  cuDeviceGetPCIBusId
        

The above will show the single call to `add`. Your timing may vary depending on the GPU allocated to you by Colab. To see the current GPU allocated to you run the following cell and look in the `Name` column where you might see, for example `Tesla T4`:

In [24]:
%%shell

nvidia-smi

Fri Nov 15 13:46:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Let's make it faster with parallelism.

# Picking up the Threads

Now that you've run a kernel with one thread that does some computation, how do you make it parallel? The key is in CUDA's `<<<1, 1>>>` syntax. This is called the execution configuration, and it tells the CUDA runtime how many parallel threads to use for the launch on the GPU. There are two parameters here, but let's start by changing the second one: the number of threads in a thread block. CUDA GPUs run kernels using blocks of threads that are a multiple of 32 in size, so 256 threads is reasonable size to choose.

```cpp
add<<<1, 256>>>(N, x, y);
```

If i run the code with only this change, it will do the computation once per thread, rather than the spreading the computation across the parallel threads. To do it properly, i need to modify the kernel. CUDA C++ provides keywords that let kernels get the indices of the running threads. Specifically, `threadIdx.x` contains the index of the current thread within its block, and `blockDim.x` contains the number of threads in the block. I'll just modify the loop to stride thro

In [1]:
%%writefile MatrixMul.cu
#include <iostream>
#include <cuda_runtime.h>

#define TILE_WIDTH 16  // Define the tile width for tiling

// Kernel function for tiled matrix multiplication
__global__ void matrixMultiplyTiled(float *d_M, float *d_N, float *d_P, int M_rows, int M_cols, int N_cols) {
    // Shared memory arrays for tiles
    __shared__ float Mds[TILE_WIDTH][TILE_WIDTH];
    __shared__ float Nds[TILE_WIDTH][TILE_WIDTH];

    // Calculate row and column indices of the element in P
    int Row = blockIdx.y * TILE_WIDTH + threadIdx.y;
    int Col = blockIdx.x * TILE_WIDTH + threadIdx.x;

    float Pvalue = 0.0;

    // Loop over all tiles
    for (int ph = 0; ph < (M_cols + TILE_WIDTH - 1) / TILE_WIDTH; ++ph) {
        // Load M and N tiles into shared memory
        if (Row < M_rows && ph * TILE_WIDTH + threadIdx.x < M_cols) {
            Mds[threadIdx.y][threadIdx.x] = d_M[Row * M_cols + ph * TILE_WIDTH + threadIdx.x];
        } else {
            Mds[threadIdx.y][threadIdx.x] = 0.0;
        }

        if (ph * TILE_WIDTH + threadIdx.y < M_cols && Col < N_cols) {
            Nds[threadIdx.y][threadIdx.x] = d_N[(ph * TILE_WIDTH + threadIdx.y) * N_cols + Col];
        } else {
            Nds[threadIdx.y][threadIdx.x] = 0.0;
        }

        // Synchronize threads to ensure all threads have loaded their tiles
        __syncthreads();

        // Compute partial product for the tile
        for (int k = 0; k < TILE_WIDTH; ++k) {
            Pvalue += Mds[threadIdx.y][k] * Nds[k][threadIdx.x];
        }

        // Synchronize threads to ensure all threads are done computing before loading the next tile
        __syncthreads();
    }

    // Write the computed value to the output matrix
    if (Row < M_rows && Col < N_cols) {
        d_P[Row * N_cols + Col] = Pvalue;
    }
}

void matrixMultiply(float *h_M, float *h_N, float *h_P, int M_rows, int M_cols, int N_cols) {
    float *d_M, *d_N, *d_P;

    // Allocate device memory
    cudaMalloc(&d_M, M_rows * M_cols * sizeof(float));
    cudaMalloc(&d_N, M_cols * N_cols * sizeof(float));
    cudaMalloc(&d_P, M_rows * N_cols * sizeof(float));

    // Copy data to device
    cudaMemcpy(d_M, h_M, M_rows * M_cols * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_N, h_N, M_cols * N_cols * sizeof(float), cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    dim3 dimBlock(TILE_WIDTH, TILE_WIDTH);
    dim3 dimGrid((N_cols + TILE_WIDTH - 1) / TILE_WIDTH, (M_rows + TILE_WIDTH - 1) / TILE_WIDTH);

    // Launch kernel
    matrixMultiplyTiled<<<dimGrid, dimBlock>>>(d_M, d_N, d_P, M_rows, M_cols, N_cols);

    // Copy result back to host
    cudaMemcpy(h_P, d_P, M_rows * N_cols * sizeof(float), cudaMemcpyDeviceToHost);

    // Free device memory
    cudaFree(d_M);
    cudaFree(d_N);
    cudaFree(d_P);
}

int main() {
    // Define matrix dimensions
    int M_rows = 32, M_cols = 64, N_cols = 32;

    // Allocate and initialize host matrices
    float *h_M = new float[M_rows * M_cols];
    float *h_N = new float[M_cols * N_cols];
    float *h_P = new float[M_rows * N_cols];

    // Initialize matrices with random values
    for (int i = 0; i < M_rows * M_cols; ++i) h_M[i] = static_cast<float>(rand()) / RAND_MAX;
    for (int i = 0; i < M_cols * N_cols; ++i) h_N[i] = static_cast<float>(rand()) / RAND_MAX;

    // Perform matrix multiplication
    matrixMultiply(h_M, h_N, h_P, M_rows, M_cols, N_cols);

    // Display result
    std::cout << "Matrix multiplication completed successfully." << std::endl;

    // Free host memory
    delete[] h_M;
    delete[] h_N;
    delete[] h_P;

    return 0;
}


Writing MatrixMul.cu


In [2]:
%%shell

nvcc MatrixMul.cu -o MatrixMul
./MatrixMul

Matrix multiplication completed successfully.
